<a href="https://www.kaggle.com/code/davideliu/10000m-data-analysis?scriptVersionId=111854896" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Italian Athletics 10000m Historical Data Analysis

What's the fastest time among all 10000m athletes every year?
What time do I need to be in the top-100?
Which age do athletes reach their peak performance level? 

This notebook not only lead you to the answer of the above questions, but it also provides many useful insights and statistics through friendly data visualization charts about the 10000m performance of athletes running for the Italian Federation over the time since 2005.

You can also find the original dataset and more info about it on [Kaggle](https://www.kaggle.com/datasets/davideliu/italian-athletics-historical-best-performance).

## Initialization

Load dataset and useful libraries

In [1]:
import numpy as np
import pandas as pd
import datetime

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename == 'athletics_IT.csv':
            df_path = os.path.join(dirname, filename)

Read the dataset and visualize raw data.

In [2]:
df_name = 'athletics_IT.csv'
df = pd.read_csv(df_path, encoding='utf8').reset_index(drop=True)
df.info()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 902282 entries, 0 to 902281
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        902282 non-null  object 
 1   wind        154345 non-null  object 
 2   name        902282 non-null  object 
 3   birth-year  902279 non-null  float64
 4   team        902282 non-null  object 
 5   position    902282 non-null  int64  
 6   location    902282 non-null  object 
 7   date        902282 non-null  object 
 8   sex         902282 non-null  object 
 9   event       902282 non-null  object 
 10  type        902282 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 75.7+ MB


## Filter dataset by event

The new dataset contains the historical yearly personal best (PB) of all 10000m athletes from 2005 to 2021.

In [3]:
analyze_event = '10000m'
df = df[df.event == analyze_event]
df

,time,wind,name,birth-year,team,position,location,date,sex,event,type
526951,00:33:05,NaN,GIAMBANCO Franco,1983.0,RUNNER TEAM 99 SBV,9,Novara,2005-04-24,M,10000m,P
526952,00:28:39.260000,NaN,BATTOCLETTI Giuliano,1975.0,CO-VER SPORTIVA MAPEI,1,Vigna Di Valle,2005-05-22,M,10000m,P
526953,00:28:56.630000,NaN,PERTILE Ruggero,1974.0,ASSINDUSTRIA SPORT PD,2,Vigna Di Valle,2005-05-22,M,10000m,P
526954,00:29:02.740000,NaN,CURZI Denis,1975.0,C.S. CARABINIERI SEZ. ATLETICA,1,Rubiera,2005-04-17,M,10000m,P
526955,00:29:04,NaN,LEONE Maurizio,1973.0,C.S. CARABINIERI SEZ. ATLETICA,2,Rubiera,2005-04-17,M,10000m,P
...,...,...,...,...,...,...,...,...,...,...,...
537081,00:57:28,NaN,OLIVIERI Chiara,1989.0,TEAM BASILE,7,Castrovillari,2021-04-17,F,10000m,P
537082,00:59:00.900000,NaN,DE MASI Maria,1960.0,A.S.C.D. SILVANO FEDI,14,Santa Maria A Monte,2021-04-18,F,10000m,P
537083,01:04:19,NaN,SANSONE Patrizia,1965.0,G.S. ATHLOS MATERA,2,Potenza Matera,2021-04-18,F,10000m,P
537084,01:04:43.400000,NaN,ARRIGHI Paola,1962.0,ATLETICA CASCINA,15,Santa Maria A Monte,2021-04-18,F,10000m,P


## Preprocess the dataset;

The main operations include:
- Define a new column `age` corresponding to the age of each athlete according the year of their PB.
- Define a new column `year` (`int`) from `date` (`pd.DateTime`).
- Remove athletes whose age is < 16 years old. Those values are outliers since the data should only cover seniores categories.
- Define some `groupby` objects useful to process the data to be visualized.

In [4]:
df['date'] = pd.to_datetime(df['date'])
df['year'] = pd.to_datetime(df['date']).dt.year
df['year'] = pd.to_numeric(df['year'], downcast='integer').astype(int)
n_years = len(df['year'].unique())
df['time'] = pd.to_datetime(df['time'])
df['birth-year'].fillna(df['birth-year'].mean(), inplace=True)
df['age'] = (df['year'] - df['birth-year']).astype(int)
df = df[df['age'] >= 16]
group_ys = df.groupby([df.year, df.sex])
group_as = df.groupby([df.age, df.sex])

## Charts wrappers with Plotly

In [5]:
import plotly.express as px
import plotly.graph_objects as go


def line_plot(df, x, y, hover_data=None, color=None, title=None):
    color_discrete_map = None
    if color == 'sex':
        color_discrete_map = {'M': 'blue', 'F': 'red'}
    fig = px.line(df, x=x, y=y, hover_data=hover_data, color=color,
                  color_discrete_map=color_discrete_map, title=title)
    fig.update_yaxes(tickformat="%M'%S.%L")
    fig.update_traces(mode="markers+lines")
    return fig


def scatter_plot(df, x, y, hover_data=None, color=None, title=None):
    color_discrete_map = None
    if color == 'sex':
        color_discrete_map = {'M': 'blue', 'F': 'red'}
    fig = px.scatter(df, x=x, y=y, hover_data=hover_data, color=color,
                     color_discrete_map=color_discrete_map, title=title)
    fig.update_yaxes(tickformat="%M'%S.%L")
    return fig


def histogram_plot(df, x, y=None, hover_data=None, color=None, cumulative=False, nbins=None, title=None):
    color_discrete_map = None
    if color == 'sex':
        color_discrete_map = {'M': 'blue', 'F': 'red'}
    fig = px.histogram(df, x=x, y=y, hover_data=hover_data, color=color, color_discrete_map=color_discrete_map,
                       cumulative=cumulative, nbins=nbins, title=title)
    fig.update_yaxes(tickformat="%M'%S.%L")
    fig.update_xaxes(tickformat="%M'%S.%L")
    fig.update_layout(yaxis_title="")
    return fig


def split_violin_plot(df, x, y, split, colors=None, title=None, points='outliers'):
    assert points in ['all', 'outliers', False]
    values = df[split].unique()
    assert len(values) == 2
    if not colors:
        colors = ['blue', 'red']
    fig = go.Figure()
    for i, v in enumerate(values):
        fig.add_trace(go.Violin(x=df[x][df[split] == v],
                            y=df[y][df[split] == v],
                            legendgroup=v, scalegroup=v, name=v,
                            side='negative' if i == 0 else 'positive',
                            line_color=colors[i])
                 )
    fig.update_traces(meanline_visible=True, points=points)
    fig.update_layout(violingap=0, violinmode='overlay', title=title, legend_title=split)
    fig.update_yaxes(tickformat="%M'%S.%L")
    return fig


def multi_line_plot(x, y, hover_data=None, color=None, title=None, names=None, dash=None, legend_title=None):
    fig = go.Figure()
    if not legend_title:
        legend_title = 'category'
    for i, y_ in enumerate(y):
        fig.add_trace(go.Scatter(x=x, y=y_, name=names[i], legendrank=i+1,
                                 line=dict(color=color[i],
                                           dash=dash[i] if dash is not None else None)))
    fig.update_layout(title=title, legend_title=legend_title)
    fig.update_yaxes(tickformat="%M'%S.%L")
    return fig

# Data visualization

In [6]:
top1 = group_ys.head(1)
fig = line_plot(top1, x='year', y="time", hover_data=['name', 'team'], color='sex',
                title='Historical top-1 time - {}'.format(analyze_event))
fig.show()

In [7]:
df_sub50 = df.set_index('time').between_time('00:00:00', '00:50:00').reset_index()
fig = split_violin_plot(df_sub50, x='year', y="time", split='sex', points=False, 
                        title='Historical time distribution - {}'.format(analyze_event))
fig.show()

In [8]:
time_top100 = group_ys.head(100)
time_top100_sub50 = df.set_index('time').between_time('00:00:00', '00:50:00').reset_index()
fig = split_violin_plot(time_top100_sub50, x='year', y="time", split='sex',
                        title='Historical top-100 time distribution - {}'.format(analyze_event))
fig.show()

In [9]:
thr_top15 = group_ys.nth(15)['time'].reset_index()
thr_top15_m = thr_top15[thr_top15.sex == 'M']
thr_top30 = group_ys.nth(30)['time'].reset_index()
thr_top30_m = thr_top30[thr_top30.sex == 'M']
thr_top15 = group_ys.nth(15)['time'].reset_index()
thr_top15_f = thr_top15[thr_top15.sex == 'F']
thr_top30 = group_ys.nth(30)['time'].reset_index()
thr_top30_f = thr_top30[thr_top30.sex == 'F']
fig = multi_line_plot(x=thr_top30_m.year, 
                      y=[thr_top30_m.time, thr_top30_f.time, thr_top15_m.time, thr_top15_f.time],
                      color=['blue', 'red', 'blue', 'red'],
                      dash=['dash', 'dash', 'dot', 'dot'],
                      names=['Top-30 M', 'Top-15 F', 'Top-30 M', 'Top-15 F'],
               title='Historical top-15 and top-30 entry time - {}'.format(analyze_event))
fig.show()

In [10]:
df['date_loc'] = df.date.astype(str) + df.location.astype(str)
races = df.groupby(df.year)['date_loc'].nunique().reset_index()
races.rename(columns={'date_loc': 'races'}, inplace=True)
fig = histogram_plot(races, x='year', y="races", nbins=n_years * 2 + 1, hover_data=['year', 'races'],
                     title='Historical number of races - {}'.format(analyze_event))
fig.show()

In [11]:
participants = group_ys['time'].count().reset_index()
participants.rename(columns={'time': 'athletes'}, inplace=True)
fig = histogram_plot(participants, x='year', y="athletes", color='sex', nbins=n_years * 2 + 1, hover_data=['year', 'athletes'],
                     title='Historical number of athletes - {}'.format(analyze_event))
fig.show()

- There is a sharp increase in the number of participants in 2014, especially for male athletes. Given the results of the previous chart, the increase is not due to the availability of more races but probably caused by more athletes registered in the system.

In [12]:
mean_age = group_as['time'].mean().reset_index()
mean_age_sub55 = mean_age.set_index('time').between_time('00:00:00', '00:55:00').reset_index()
fig = scatter_plot(mean_age_sub55, x='age', y="time", color='sex', hover_data=['age'],
                   title='Mean time by age - {}'.format(analyze_event))
fig.show()

In [13]:
event_pb = df.loc[df.groupby([df.name, df.sex]).time.idxmin()]
event_pb['count(%)'] = event_pb['age'] / event_pb.groupby(['age', 'sex'])['age'].transform('sum') * 100
event_pb_top100 = event_pb.sort_values(['time'], ascending=True).groupby(event_pb.sex).head(100)
fig = scatter_plot(event_pb_top100, x='age', y="time", color='sex', hover_data=['time', 'name', 'team', 'age'],
                   title='Age of top-100 athletes PB - {}'.format(analyze_event))
fig.show()

In [14]:
n_ages = len(event_pb_top100.age.unique())
fig = histogram_plot(event_pb_top100, x='age', color='sex', nbins=n_ages * 3 + 1, hover_data=['age'],
                     title='Number of top-100 athletes PB by age - {}'.format(analyze_event))
fig.show()

In [15]:
fig = histogram_plot(event_pb, x='age', color='sex', nbins=200, hover_data=['age'],
                     title='Number of athletes PB by age - {}'.format(analyze_event))
fig.show()

In [16]:
event_pb_limit = event_pb.set_index('time').between_time('00:00', '00:50:00').reset_index()
fig = histogram_plot(event_pb_limit, x='time', color='sex', cumulative=True, nbins=100, hover_data=['time'],
                     title='Number of athletes PB under different time thresholds - {}'.format(analyze_event))
fig.show()

In [17]:
df_sort_time = df.sort_values(['time'], ascending=True)
df_filter = df_sort_time.drop_duplicates(subset=['name'], keep="first")  # remove duplicates keep fastest time
df_team_count = df_filter.groupby([df_filter.team]).size().reset_index(name='athletes')
df_team_count = df_team_count.sort_values(['athletes'], ascending=False).head(20)
fig = histogram_plot(df_team_count, x='team', y='athletes', hover_data=['athletes'],
                     title='Number of athletes by team - {}'.format(analyze_event))
fig.show()

In [18]:
df_team_time = df_filter.groupby(df_filter.team)['time'].mean().reset_index()
df_team_time = df_team_time.sort_values(['time'], ascending=True).head(20)
fig = line_plot(df_team_time, x='team', y='time',
                title='Mean athletes PB by team - {}'.format(analyze_event))
fig.show()